In [ ]:
import numpy as np
import pandas as pd
import altair as alt

In [63]:
# Source for data in `data/prices/malaga.yaml`:
# https://www.emasa.es/wp-content/uploads/2020/06/Factura-por-habitante.pdf

# Source https://www.emasa.es/wp-content/uploads/2020/06/Ejemplo-Factura-por-Habitante.pdf
fix_costs = {
    'abastecimiento': 2.547,
    'depuracion': 0.557
}

In [272]:
# Let's create a function to calculate the overall cost as a function
# Of usage and users
# This works as follows:
# For each category, calculate the total cost per user.
# Note that the blocks signify min and max usage levels,
# So we want to sum up over all blocks that are relevant.

group = "desalacion"
usage_per_user = 10
large_number = 100000

def group_cost(data, group, usage_per_user):
    cost = 0
    usage_remaining = usage_per_user
    for category in data[group]:
        for block in data[group]:
            brange = block['usage_range']
            min_usage = brange[0]
            max_usage = brange[1] or large_number
            block_range = max_usage - min_usage
            if usage_remaining <= 0:
                break
            max_usage_in_block = min(usage_remaining, block_range)
            cost += block['price_per_m3'] * max_usage_in_block
            usage_remaining -= max_usage_in_block
    return cost

def total_variable_cost(data, usage_per_user):
    cost = 0
    for group in data.keys():
        cost += group_cost(data, group, usage_per_user)
    return cost

total_fix_costs = sum(fix_costs.values())
def all_costs(data, usage_per_user, total=False):
    total_variable_costs = total_variable_cost(data, usage_per_user)
    costs = {
        'fix': total_fix_costs,
        'variable': total_variable_costs,
    }
    return costs

def all_costs_linear(constant_cost):
    def get_costs(data, usage_per_user):
        costs = {
            'fix': total_fix_costs,
            'variable': usage_per_user * constant_cost
        }
        return costs
    return get_costs

functions = {
    'current': all_costs
}

vals_linear = np.arange(1, 40, 0.4)
for val in vals_linear:
    functions[f'linear_{val:02.0f}'] = all_costs_linear(val/10)

usage = np.arange(0, 50, 0.1)

df = pd.DataFrame({'usage': usage})

for cost_type in functions.keys():
    df['costs_' + cost_type] = df['usage'].apply(lambda x: np.sum(list(functions[cost_type](data, x).values())))
    df['marginal_cost_' + cost_type] = df['costs_' + cost_type].diff()

df = df.query('usage > 0').copy()
df['full'] = np.round(df.usage) == df.usage

In [273]:

# Show the same graph using altair
# First turn the data into long format
df_long = df.query('usage <= 6').melt(id_vars='usage', value_vars=['marginal_cost_' + cost_type for cost_type in functions.keys()])
df_long['type'] = df_long['variable'].apply(lambda x: x.replace('marginal_cost_', ''))

chart = alt.Chart(df_long).mark_line().encode(
    x='usage',
    y='value',
    color='type'
)
chart

alt.Chart(...)

In [275]:
const_baseline = 1
elasticity = -1

num_obs = 100
results = []
for i in range(num_obs):
    res = {}
    randvar = np.random.rand() 
    const_random = const_baseline * np.exp(randvar*5)
    res['const'] = const_random
    demand = const_random * np.exp(elasticity * usage)
    df['demand'] = const_random * np.exp(elasticity * df.usage)
    
    for supply_type in functions.keys():
        indx = df.query(f'marginal_cost_{supply_type} > demand').index.min()
        quantity = df.loc[indx, 'usage']
        res[f'quantity_{supply_type}'] = quantity
        res[f'costs_{supply_type}'] = df.loc[indx, f'costs_{supply_type}']
    results.append(res)

df_quantities = pd.DataFrame(results)

sums = df_quantities.mean()
columns = ['quantity', 'costs']
df_wide = pd.DataFrame({col: sums.filter(like=col).values for col in columns})
df_wide.index = sums.filter(like='quantity').index.str.replace('quantity_', '')
df_wide

,quantity,costs
current,4.139,8.115879
linear_01,6.796,4.055440
linear_02,6.350,4.501000
linear_03,5.906,5.112040
linear_04,5.697,5.496740
linear_05,5.450,6.047000
linear_06,5.306,6.393720
linear_07,5.134,6.903160
linear_08,5.031,7.229420
linear_09,4.893,7.703420


,quantity,costs
linear_05,564.9,592.8500
linear_10,496.3,806.7000
current,424.7,823.2439
